##Montar Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install rdflib
!pip install torch-geometric -q

# Semillas para que sea replicable

In [ ]:
import random
import numpy as np
import torch

def set_seed(seed=42):
    random.seed(seed)                      # Semilla para random
    np.random.seed(seed)                   # Semilla para NumPy
    torch.manual_seed(seed)                # Semilla para PyTorch
    torch.cuda.manual_seed(seed)           # Para CUDA (si usás GPU)
    torch.cuda.manual_seed_all(seed)       # Por si hay múltiples GPUs
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)  # Elegí el número que quieras


## Ruta archivo

In [ ]:
# Ruta al archivo .ttl que subiste
ttl_path = "/content/drive/MyDrive/Proyecto_Grafos_Conocimiento/grafoA_metadata.ttl"

## Construcción de Heterodata

In [ ]:
# script_export_to_heterodata.py

from rdflib import Graph, URIRef, RDF, Literal
from collections import defaultdict
from torch_geometric.data import HeteroData
import torch
from datetime import datetime

# 1) Carga del grafo RDF
#ttl_file = "grafo_completo.ttl"  # Ajusta la ruta a la propia
ttl_file = ttl_path
g = Graph()
g.parse(ttl_file, format="turtle")

# 2) Helper de namespace
NS = "http://example.org/cdph/"
def uri(prop): return URIRef(NS + prop)

# 3) Definición de clases RDF
rdf_types = {
    "Product":     uri("Product"),
    "Brand":       uri("Brand"),
    "Company":     uri("Company"),
    "Category":    uri("Category"),
    "SubCategory": uri("SubCategory"),
    "Chemical":    uri("Chemical"),
}

# 4) Indexar nodos por tipo
node_maps  = {nt: {} for nt in rdf_types}
node_counts = {nt: 0 for nt in rdf_types}
for subj, _, obj in g.triples((None, RDF.type, None)):
    for ntype, cls in rdf_types.items():
        if obj == cls and subj not in node_maps[ntype]:
            node_maps[ntype][subj] = node_counts[ntype]
            node_counts[ntype] += 1

# 5) Pre-calcular edad y etiqueta para productos, y conteo de químicos
prod_age   = {}
prod_label = {}
chem_counts = defaultdict(int)

for prod_node in node_maps["Product"]:
    # Edad
    init = g.value(prod_node, uri("hasInitialDateReported"))
    most = g.value(prod_node, uri("hasMostRecentDateReported"))
    age_days = 0
    if isinstance(init, Literal) and isinstance(most, Literal):
        try:
            d0 = datetime.fromisoformat(str(init))
            d1 = datetime.fromisoformat(str(most))
            age_days = (d1 - d0).days
        except:
            pass
    prod_age[prod_node] = age_days
    # Etiqueta
    prod_label[prod_node] = int(bool(g.value(prod_node, uri("hasDiscontinuedDate"))))

# Conteo de químicos por producto
for s, _, o in g.triples((None, uri("productHasChemical"), None)):
    if s in node_maps["Product"] and o in node_maps["Chemical"]:
        chem_counts[s] += 1

# 6) Construir HeteroData
data = HeteroData()

# 7) Features y labels para Product
prod_feats  = []
prod_labels = []
for prod_node, idx in sorted(node_maps["Product"].items(), key=lambda x: x[1]):
    feat = [
        chem_counts.get(prod_node, 0),         # número de químicos
        int(bool(g.value(prod_node, uri("productHasBrand")))),
        int(bool(g.value(prod_node, uri("productHasCategory")))),
        int(bool(g.value(prod_node, uri("productHasSubCategory")))),
        int(bool(g.value(prod_node, uri("productMadeByCompany")))),
    ]
    prod_feats.append(feat)
    prod_labels.append(prod_label[prod_node])

data["Product"].x = torch.tensor(prod_feats, dtype=torch.float)
data["Product"].y = torch.tensor(prod_labels, dtype=torch.long)

# 8) Features para Chemical
chem_feats = []
for chem_node, idx in sorted(node_maps["Chemical"].items(), key=lambda x: x[1]):
    hv = 0
    h_lit = g.value(chem_node, uri("hasHazardScore"))
    if isinstance(h_lit, Literal):
        py = h_lit.toPython()
        if isinstance(py, (int, float)):
            hv = int(py)
        else:
            s = str(py)
            hv = int(s) if s.isdigit() else 0
    flags = [ # Me di cuenta de que son flags solo son indicaciones booleans despues de hacer la GNN, es decir, no estamos usando
            # flags como tal, sino que son indicadores de si existen o no... lo cual como todos tiene hacen esta features inutiles XD.
            # *Necesita cambio*
        int(bool(g.value(chem_node, uri("AllergiesConcern")))),
        int(bool(g.value(chem_node, uri("CancerConcern")))),
        int(bool(g.value(chem_node, uri("DevelopReproductiveConcern")))),
        int(bool(g.value(chem_node, uri("UseRestrictionsConcern")))),
    ]
    chem_feats.append([hv] + flags)

data["Chemical"].x = torch.tensor(chem_feats, dtype=torch.float)

# 9) Features “agregadas” para Brand, Company, Category, SubCategory
def build_hub_features(node_type, prop):
    feats = []
    for node, idx in sorted(node_maps[node_type].items(), key=lambda x: x[1]):
        prods = [s for s, _, o in g.triples((None, uri(prop), node))]
        n = len(prods)
        if n > 0:
            avg_age  = sum(prod_age[p] for p in prods) / n
            avg_chem = sum(chem_counts.get(p,0) for p in prods) / n
        else:
            avg_age = avg_chem = 0.0
        feats.append([n, avg_age, avg_chem])
    return torch.tensor(feats, dtype=torch.float)

data["Brand"].x       = build_hub_features("Brand",       "productHasBrand")
data["Company"].x     = build_hub_features("Company",     "productMadeByCompany")
data["Category"].x    = build_hub_features("Category",    "productHasCategory")
data["SubCategory"].x = build_hub_features("SubCategory", "productHasSubCategory")

# 10) Extraer aristas
obj_props = {
    ("Product", "hasBrand",       "Brand"):       "productHasBrand",
    ("Product", "hasCategory",    "Category"):    "productHasCategory",
    ("Product", "hasSubCategory", "SubCategory"): "productHasSubCategory",
    ("Product", "madeBy",         "Company"):     "productMadeByCompany",
    ("Product", "hasChemical",    "Chemical"):    "containsChemical",
}

for (src, rel, dst), prop in obj_props.items():
    edges = []
    for s, _, o in g.triples((None, uri(prop), None)):
        if s in node_maps[src] and o in node_maps[dst]:
            edges.append((node_maps[src][s], node_maps[dst][o]))
    if edges:
        data[(src, rel, dst)].edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

# 11) Validar y mostrar resumen
data.validate(raise_on_error=True)
print(data)

# 11) Guardar HeteroData en un archivo .pt dentro de Drive
torch.save(data, "/content/drive/MyDrive/Proyecto_Grafos_Conocimiento/hetero_data.pt")
print("HeteroData serializado en '/content/drive/MyDrive/Proyecto_Grafos_Conocimiento/hetero_data.pt'")

# TRAINING EMBEDDINGS

In [ ]:
import torch
from torch_geometric.data import HeteroData
from torch_geometric.data.storage import NodeStorage, EdgeStorage, BaseStorage

# Permitir todas las clases necesarias para cargar el HeteroData
with torch.serialization.safe_globals({
    NodeStorage: NodeStorage,
    EdgeStorage: EdgeStorage,
    BaseStorage: BaseStorage
}):
    data = torch.load("/content/drive/MyDrive/Proyecto_Grafos_Conocimiento/hetero_data.pt")

# Mostrar resumen del grafo cargado
print(data)
print("Tipos de nodos:", data.node_types)
print("Tipos de aristas:", data.edge_types)


In [ ]:
edge_keys = list(data.edge_index_dict.keys())  # Copia para evitar errores al iterar
for (src, rel, dst) in edge_keys:
    inv_rel = rel + "_rev"
    if (dst, inv_rel, src) not in data.edge_index_dict:
        data[(dst, inv_rel, src)].edge_index = data[(src, rel, dst)].edge_index.flip(0)

In [ ]:
from torch_geometric.nn import MetaPath2Vec

# 🔁 Paso 1: Definir un solo metacaminos válido
metapath = [
    ('Product', 'hasBrand', 'Brand'),
    ('Brand', 'hasBrand_rev', 'Product')
]

# ⚙️ Paso 2: Crear el modelo con argumentos en orden correcto
model = MetaPath2Vec(
    data.edge_index_dict,  # edge_index_dict: diccionario de aristas
    128,                   # embedding_dim: tamaño de embedding
    metapath,              # metapath: lista de relaciones conectadas
    4,                     # walk_length: largo de caminata
    2                      # context_size: tamaño de ventana
)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

# Cargar generador de muestras para entrenamiento
loader = model.loader(batch_size=128, shuffle=True, num_workers=0)

# Usamos Adam ya que los embeddings son densos
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        pos_rw, neg_rw = pos_rw.to(device), neg_rw.to(device)
        loss = model.loss(pos_rw, neg_rw)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    return total_loss / len(loader)

In [ ]:
train_losses = []

for epoch in range(1, 35):  # Aumente si quiere más calidad
    loss = train()
    train_losses.append(loss)
    print(f"Época {epoch:02d} | Pérdida: {loss:.4f}")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,4))
plt.plot(train_losses, marker='o', color='blue', label="Pérdida de entrenamiento")
plt.title("Curva de pérdida durante el entrenamiento")
plt.xlabel("Época")
plt.ylabel("Pérdida")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Ruta donde guardar el modelo entrenado
ruta_modelo = "/content/drive/MyDrive/Proyecto_Grafos_Conocimiento/metapath2vec_model.pt"

# Guardar modelo completo
torch.save(model.state_dict(), ruta_modelo)
print(f"📦 Modelo guardado en: {ruta_modelo}")

In [ ]:
model.eval()
with torch.no_grad():
    product_emb = model('Product').cpu().numpy()  # tensor → numpy array

In [ ]:
from sklearn.preprocessing import normalize

# Normalizar cada embedding al vector unitario
product_emb_normalized = normalize(product_emb)

# Luego aplicás t-SNE, PCA o UMAP sobre esto

 Opción A: Visualizar con PCA (Análisis de Componentes Principales

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

pca = PCA(n_components=2)
emb_pca = pca.fit_transform(product_emb)

plt.figure(figsize=(6,6))
plt.scatter(emb_pca[:,0], emb_pca[:,1], s=5, alpha=0.7)
plt.title("Embeddings de Productos (PCA)")
plt.xlabel("Componente 1")
plt.ylabel("Componente 2")
plt.grid(True)
plt.show()

In [ ]:
import plotly.express as px
import pandas as pd

# Transformar a PCA
emb_pca = PCA(n_components=2).fit_transform(product_emb)

# Crear DataFrame con etiquetas opcionales
df = pd.DataFrame({
    "x": emb_pca[:, 0],
    "y": emb_pca[:, 1],
    "id": [f"Producto_{i}" for i in range(len(emb_pca))]  # etiquetas opcionales
})

# Gráfico interactivo
fig = px.scatter(df, x="x", y="y", text="id",
                 title="Visualización interactiva de embeddings (PCA)",
                 labels={"x": "Componente 1", "y": "Componente 2"})

fig.update_traces(marker=dict(size=4), textposition='top center')
fig.update_layout(height=600, width=600)
fig.show()

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # para gráfico 3D

# Aplicar PCA a 3 componentes
pca = PCA(n_components=3)
emb_pca_3d = pca.fit_transform(product_emb)

# Visualizar en 3D
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(
    emb_pca_3d[:,0],
    emb_pca_3d[:,1],
    emb_pca_3d[:,2],
    s=5,
    alpha=0.7,
    color='steelblue'
)

ax.set_title("Embeddings de Productos (PCA 3D)")
ax.set_xlabel("PC 1")
ax.set_ylabel("PC 2")
ax.set_zlabel("PC 3")
plt.tight_layout()
plt.show()

In [ ]:
explained = pca.explained_variance_ratio_
print(f"📊 Varianza explicada por cada componente: {explained}")
print(f"📈 Varianza total explicada (3 componentes): {explained.sum():.2%}")

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Asegúrate de tener los embeddings en esta variable (sin gradientes)
# product_emb = model('Product') si no lo has hecho antes
model.eval()
with torch.no_grad():
    product_emb = model('Product').cpu().numpy()

# Aplicar t-SNE (puedes ajustar `perplexity` y `n_iter` si quieres mejorar resultados)
tsne = TSNE(n_components=2, perplexity=30, n_iter=350, random_state=42)
product_emb_2d = tsne.fit_transform(product_emb)

# Graficar
plt.figure(figsize=(10, 8))
plt.scatter(product_emb_2d[:, 0], product_emb_2d[:, 1], s=5, alpha=0.6)
plt.title("📌 Visualización de Embeddings de Productos (t-SNE 2D)")
plt.xlabel("Componente 1")
plt.ylabel("Componente 2")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np

norms = np.linalg.norm(product_emb, axis=1)
print(f"🔍 Máxima norma: {norms.max():.4f}, Mínima norma: {norms.min():.4f}")

In [ ]:
norms = product_emb.norm(dim=1)
print(f"Máxima norma: {norms.max():.4f}, Mínima norma: {norms.min():.4f}")


In [ ]:
!pip install umap-learn

In [ ]:
import umap.umap_ as umap
import matplotlib.pyplot as plt

# Supongamos que product_emb es el tensor de embeddings de productos
# Asegúrate de pasarlo como NumPy
product_emb_np = product_emb.cpu().detach().numpy()

# UMAP en 2D
umap_2d = umap.UMAP(n_components=2, random_state=42)
emb_umap = umap_2d.fit_transform(product_emb_np)

# Visualización
plt.figure(figsize=(10, 8))
plt.scatter(emb_umap[:, 0], emb_umap[:, 1], s=2, alpha=0.6)
plt.title("Embeddings de Productos (UMAP 2D)", fontsize=14)
plt.xlabel("UMAP 1")
plt.ylabel("UMAP 2")
plt.grid(True)
plt.show()

# Opcion con metacaminos diferentes y relaciones inversas para captar más contexto y simetras, además de saltos más largos

In [ ]:
!pip install rdflib
!pip install torch-geometric -q

In [ ]:
import torch
from torch_geometric.data import HeteroData
from torch_geometric.data.storage import NodeStorage, EdgeStorage, BaseStorage

# Permitir todas las clases necesarias para cargar el HeteroData
with torch.serialization.safe_globals({
    NodeStorage: NodeStorage,
    EdgeStorage: EdgeStorage,
    BaseStorage: BaseStorage
}):
    data = torch.load("/content/drive/MyDrive/Proyecto_Grafos_Conocimiento/hetero_data.pt")

# Mostrar resumen del grafo cargado
print(data)
print("Tipos de nodos:", data.node_types)
print("Tipos de aristas:", data.edge_types)

In [ ]:
for (src, rel, dst), edge in data.edge_index_dict.items():
    if src == 'Product':
        print(f"{rel} → {dst}: {edge.size(1)} conexiones")


In [ ]:
print(data.edge_index_dict.keys())


In [ ]:
print(data.edge_index_dict.keys())


In [ ]:
# 1. Generar relaciones inversas para cada edge
edge_keys = list(data.edge_index_dict.keys())
for (src, rel, dst) in edge_keys:
    inv_rel = rel + "_rev"
    if (dst, inv_rel, src) not in data.edge_index_dict:
        data[(dst, inv_rel, src)].edge_index = data[(src, rel, dst)].edge_index.flip(0)

# 2. Importar librerías necesarias
from torch_geometric.nn import MetaPath2Vec

# 3. Definir todos los metacaminos válidos basados en tu grafo
metapath = [
    ('Product', 'hasBrand', 'Brand'),
    ('Brand', 'hasBrand_rev', 'Product'),

    ('Product', 'hasCategory', 'Category'),
    ('Category', 'hasCategory_rev', 'Product'),

    ('Product', 'hasSubCategory', 'SubCategory'),
    ('SubCategory', 'hasSubCategory_rev', 'Product'),

    ('Product', 'madeBy', 'Company'),
    ('Company', 'madeBy_rev', 'Product'),
]

# 4. Crear el modelo con parámetros ajustables

model = MetaPath2Vec(
    data.edge_index_dict,
    embedding_dim=128,     # Buena representación
    metapath=metapath,
    walk_length=7,         # Caminata razonable
    context_size=3         # Algo más de contexto
)
# 5. Enviar a GPU si está disponible
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

# 6. Cargar generador de muestras
loader = model.loader(batch_size=128, shuffle=True, num_workers=2)

# 7. Definir optimizador y función de entrenamiento
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        pos_rw, neg_rw = pos_rw.to(device), neg_rw.to(device)
        loss = model.loss(pos_rw, neg_rw)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    return total_loss / len(loader)

# 8. Entrenar por varias épocas
train_losses = []
for epoch in range(1,201):
    loss = train()
    train_losses.append(loss)
    print(f"Época {epoch:02d} | Pérdida: {loss:.4f}")

# Guardar modelo con fecha y hora para poder cargarlo

In [ ]:
from datetime import datetime
import torch

# Obtener fecha y hora actual como string
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Se define la ruta con timestamp incluido para guardarlo
ruta_modelo = f"/content/drive/MyDrive/Proyecto_Grafos_Conocimiento/metapath2vec_model_{timestamp}.pt"

# Guardar modelo con la fecha para saber cual es en particular
torch.save(model.state_dict(), ruta_modelo)
print(f"💾 Modelo guardado en: {ruta_modelo}")


# Graficar la pérdida

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_losses, marker='*', color='red', label="Pérdida de entrenamiento")
plt.title("Curva de pérdida durante el entrenamiento")
plt.xlabel("Época")
plt.ylabel("Pérdida")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

Descendío bien, por lo que debería haber aprendido relativamente bien

In [ ]:
!pip install plotly

Obtenemos embeddings

In [ ]:
from rdflib import Graph, RDF, URIRef
g = Graph()
g.parse("/content/drive/MyDrive/Proyecto_Grafos_Conocimiento/grafoA_metadata.ttl", format="turtle")

In [ ]:
NS = "http://example.org/cdph/"
def uri(prop): return URIRef(NS + prop)

In [ ]:
import pandas as pd

name_prop = uri("hasProductName")
products = []

for subj in g.subjects(RDF.type, uri("Product")):
    label = g.value(subj, name_prop)
    name = str(label) if label else subj.split("/")[-1]
    products.append((str(subj), name))

df_products = pd.DataFrame(products, columns=["uri", "label"])
df_products = df_products.sort_values(by="uri").reset_index(drop=True)

In [ ]:
import pandas as pd

name_prop = uri("hasProductName")
products = []

for subj in g.subjects(RDF.type, uri("Product")):
    label = g.value(subj, name_prop)
    name = str(label) if label else subj.split("/")[-1]
    products.append((str(subj), name))

df_products = pd.DataFrame(products, columns=["uri", "label"])
df_products = df_products.sort_values(by="uri").reset_index(drop=True)

In [ ]:
print("Ejemplos de mapeo URI ↔ Nombre de producto:\n")
for i in range(10):  # Puedes aumentar el rango si quieres más
    print(f"{i}. {df_products.iloc[i]['uri']} → {df_products.iloc[i]['label']}")

In [ ]:
# Selecciona el URI del producto que quieres revisar
prod_uri = URIRef("http://example.org/cdph/Product-18")

# Busca todas las propiedades asociadas a ese producto
for p, o in g.predicate_objects(subject=prod_uri):
    print(f"{p} -> {o}")

In [ ]:
model.eval()
with torch.no_grad():
    product_emb = model('Product').cpu().numpy()

In [ ]:
from sklearn.preprocessing import normalize
product_emb_normalized = normalize(product_emb, norm='l2')

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
product_2d = pca.fit_transform(product_emb_normalized)

In [ ]:
import plotly.express as px

# DataFrame con los componentes principales
df_plot = pd.DataFrame(product_2d, columns=["PC1", "PC2"])

# Agregamos URI y nombre
df_plot["uri"] = df_products["uri"]
df_plot["Nombre"] = df_products["label"]

# Combinamos URI + Nombre
df_plot["uri_nombre"] = df_plot["uri"] + " → " + df_plot["Nombre"]

# Gráfico interactivo
fig = px.scatter(
    df_plot,
    x="PC1", y="PC2",
    hover_name="uri_nombre",  # Se muestra URI + nombre al pasar el mouse
    title="Visualización PCA de embeddings de productos"
)
fig.show()

In [ ]:
from sklearn.decomposition import PCA
import plotly.express as px
import pandas as pd

# 1. PCA a 3 componentes
pca_3d = PCA(n_components=3)
product_3d = pca_3d.fit_transform(product_emb_normalized)

# 2. Crear DataFrame con PC1, PC2 y PC3
df_plot3d = pd.DataFrame(product_3d, columns=["PC1", "PC2", "PC3"])

# 3. Añadir URI y nombre del producto
df_plot3d["uri"] = df_products["uri"]
df_plot3d["Nombre"] = df_products["label"]
df_plot3d["uri_nombre"] = df_plot3d["uri"] + " → " + df_plot3d["Nombre"]

# 4. Gráfico 3D interactivo
fig = px.scatter_3d(
    df_plot3d,
    x="PC1", y="PC2", z="PC3",
    hover_name="uri_nombre",
    title="Visualización PCA 3D de embeddings de productos"
)

fig.show()

In [ ]:
# Obtener categoría de cada producto
category_map = {}
for s, p, o in g.triples((None, URIRef("http://example.org/cdph/productHasCategory"), None)):
    category_map[str(s)] = str(o).split("/")[-1]  # Extrae solo el número final de la categoría

# Crear una columna de categoría
df_plot3d["Categoria"] = df_plot3d["uri"].map(category_map)

In [ ]:
import plotly.express as px

fig = px.scatter(
    df_plot3d,
    x="PC1", y="PC2",
    hover_name="uri_nombre",
    color="Categoria",  # 💡 Sigue coloreando por categoría
    title="Visualización PCA 2D de productos por categoría"
)

fig.show()

In [ ]:
fig = px.scatter_3d(
    df_plot3d,
    x="PC1", y="PC2", z="PC3",
    hover_name="uri_nombre",
    color="Categoria",  # 💡 Aquí se colorea por categoría
    title="Visualización PCA 3D de productos por categoría"
)

fig.show()

In [ ]:
!pip install umap-learn

# Se aplicó reducción de dimensionalidad en dos partes: primero con PCA a 50 componentes principales para capturar las estructuras globales más relevantes y eliminar parte del ruido, y luego con t-SNE para proyectar el espacio que da la proyección en 2D/3D, para poder explorar las relaciones locales entre productos cosméticos.

In [ ]:
from sklearn.manifold import TSNE
import umap.umap_ as umap
from sklearn.decomposition import PCA
import pandas as pd
import plotly.express as px

# DataFrame base con URI y Nombre del producto
df_base = pd.DataFrame({
    "uri": df_products["uri"],
    "uri_nombre": df_products["uri"] + " → " + df_products["label"],
    "Categoria": df_products["uri"].map(category_map)
})

# PCA para reducción previa (usado luego en t-SNE acelerado)
pca = PCA(n_components=50)
X_pca = pca.fit_transform(product_emb_normalized)

### 1. t-SNE 2D
tsne_2d = TSNE(n_components=2, perplexity=30, n_iter=250)
X_tsne_2d = tsne_2d.fit_transform(X_pca)

fig1 = px.scatter(
    x=X_tsne_2d[:, 0], y=X_tsne_2d[:, 1],
    color=df_base["Categoria"],
    hover_name=df_base["uri_nombre"],
    title="t-SNE 2D con PCA previo"
)
fig1.show()

### 2. t-SNE 3D
tsne_3d = TSNE(n_components=3, perplexity=30, n_iter=250)
X_tsne_3d = tsne_3d.fit_transform(X_pca)

fig2 = px.scatter_3d(
    x=X_tsne_3d[:, 0], y=X_tsne_3d[:, 1], z=X_tsne_3d[:, 2],
    color=df_base["Categoria"],
    hover_name=df_base["uri_nombre"],
    title="t-SNE 3D con PCA previo"
)
fig2.show()

### 3. UMAP 2D
umap_2d = umap.UMAP(n_components=2)
X_umap_2d = umap_2d.fit_transform(product_emb_normalized)

fig3 = px.scatter(
    x=X_umap_2d[:, 0], y=X_umap_2d[:, 1],
    color=df_base["Categoria"],
    hover_name=df_base["uri_nombre"],
    title="UMAP 2D"
)
fig3.show()

### 4. UMAP 3D
umap_3d = umap.UMAP(n_components=3)
X_umap_3d = umap_3d.fit_transform(product_emb_normalized)

fig4 = px.scatter_3d(
    x=X_umap_3d[:, 0], y=X_umap_3d[:, 1], z=X_umap_3d[:, 2],
    color=df_base["Categoria"],
    hover_name=df_base["uri_nombre"],
    title="UMAP 3D"
)
fig4.show()